# Script version 2.2.1 for Machine Learning Core (MLC)

## General settings

In [ ]:
import os, datetime
current_directory = os.path.join(os.getcwd(), datetime.datetime.now().strftime('%Y-%m-%d_%H-%M-%S'))
os.makedirs(current_directory)
print("Current directory: " + current_directory)
arff_filename = os.path.join(current_directory, "features.arff")
ucf_filename = os.path.join(current_directory, "MLC_configuration.ucf")

# private import:
import mlc_configurator
from mlc_configurator import *
import decision_tree_generator
from decision_tree_generator import generateDecisionTree
from decision_tree_generator import generate_subset_of_ARFF
import mlc_test
from mlc_test import *

# Unico version check: 
from win32com.client import Dispatch
ver_parser = Dispatch('Scripting.FileSystemObject')
mlc_app = "C:/Program Files (x86)/STMicroelectronics/Unico-GUI/unico.exe"  ## Windows
unico_version = "0.0.0.0"
if os.path.isfile(mlc_app):
    unico_version = ver_parser.GetFileVersion(mlc_app)
    print("Unico version: " + unico_version)
expected_version = "9.10.0.0"
if unico_version != expected_version:
    print("\nERROR: wrong Unico version. Please download version " + expected_version + " from: https://www.st.com/en/development-tools/unico-gui.html")

device_name = "LSM6DSOX"    ## list of supported devices available with mlc_configurator.get_devices()

## Step 1: Load log files, label data, define decision tree results
In this step data are loaded and results are assigned. 
Decision tree results can be configured by associating result names to result values.

In [ ]:
# Load single files
datalog1 = "../Logs/log1.txt"
result1  = "still"
datalog2 = "../Logs/log2.txt"
result2  = "walk"
datalog3 = "../Logs/log3.txt"
result3  = "fastwalk"

# Load files from a folder
#datalog1 = os.listdir("../Logs/")
#result1 = []
#for x in range(0, len(datalog1)):
    #datalog1[x] = "../Logs/" + datalog1[x]
    #result1.append("still")

datalogs = [datalog1, datalog2, datalog3]
results  = [result1, result2, result3]

# Assign values to results for each decision tree
result_names  = [] # leave empty here
result_values = [] # leave empty here
# Assign values to results for decision tree 1:
result_names.append(  [result1, result2, result3] )
result_values.append( [0,       4,       8] )
# Assign values to results for decision tree 2:
result_names.append(  [] )
result_values.append( [] )
# Assign values to results for decision tree 3:
result_names.append(  [] )
result_values.append( [] )
# Assign values to results for decision tree 4:
result_names.append(  [] )
result_values.append( [] )
# Assign values to results for decision tree 5:
result_names.append(  [] )
result_values.append( [] )
# Assign values to results for decision tree 6:
result_names.append(  [] )
result_values.append( [] )
# Assign values to results for decision tree 7:
result_names.append(  [] )
result_values.append( [] )
# Assign values to results for decision tree 8:
result_names.append(  [] )
result_values.append( [] )

dectree_filenames = []
for i in range(0,8): 
    if not result_names[i]:
        break
    else: 
        dectree_filenames.append(os.path.join(current_directory, "dectree{}.txt".format(i+1)))
n_decision_trees = i
print('n_decision_trees = %d' % (n_decision_trees))

## Step 2: Generate ARFF files
ARFF files are CSV files where the first n lines declare data types of CSV columns.
By editing lines below, we decide MLC configuration in terms of:
* filters
* features
All the features are computed within a defined time window ('window_length').
Other device settings like MLC data rate, sensor data rate and full scale are defined here.
The module 'mlc_configurator.py' is used for this step.

In [ ]:
# device settings
mlc_odr = "26 Hz"                        ## list of allowed values available with mlc_configurator.get_mlc_odr(device_name)
input_type = "accelerometer+gyroscope"   ## list of allowed values available with mlc_configurator.get_mlc_input_type(device_name)
accelerometer_fs = "2 g"                 ## list of allowed values available with mlc_configurator.get_accelerometer_fs(device_name)
accelerometer_odr = "26 Hz"              ## list of allowed values available with mlc_configurator.get_accelerometer_odr(device_name)
gyroscope_fs = "2000 dps"                ## list of allowed values available with mlc_configurator.get_gyroscope_fs(device_name)
gyroscope_odr = "26 Hz"                  ## list of allowed values available with mlc_configurator.get_gyroscope_odr(device_name)            
window_length = 52                       ## Window length (supported values: from 1 to 255)

# filters configuration
# the list of available filters can be obtained with mlc_configurator.get_filter_names(input_type)
filters_list = []   
#filters_list.append(mlc_configurator.mlc_filter("filter_1", "HP_Acc_XYZ"))
#filters_list.append(mlc_configurator.mlc_filter("filter_2", "BP_Acc_V", coef_a2=-0.5, coef_a3=0.5, coef_gain=1))
#filters_list.append(mlc_configurator.mlc_filter("filter_3", "IIR2_Acc_V", coef_b1=0.59, coef_b2=1.19, coef_b3=0.59, coef_a2=-1.01, coef_a3=0.36))

# features configuration
# full list of available features can be obtained with mlc_configurator.get_feature_names()
# full list of available inputs can be obtained with   mlc_configurator.get_mlc_inputs(device_name, input_type)
features_list = []
features_list.append(mlc_configurator.mlc_feature("MEAN", "Acc_V"))
features_list.append(mlc_configurator.mlc_feature("VARIANCE", "Acc_V"))
features_list.append(mlc_configurator.mlc_feature("ENERGY", "Acc_V"))
features_list.append(mlc_configurator.mlc_feature("PEAK_TO_PEAK", "Acc_V"))
features_list.append(mlc_configurator.mlc_feature("ZERO_CROSSING", "Acc_V", 0.5))
features_list.append(mlc_configurator.mlc_feature("POSITIVE_ZERO_CROSSING", "Acc_V", 0.5))
features_list.append(mlc_configurator.mlc_feature("NEGATIVE_ZERO_CROSSING", "Acc_V", 0.5))
features_list.append(mlc_configurator.mlc_feature("PEAK_DETECTOR", "Acc_V", 0.5))
features_list.append(mlc_configurator.mlc_feature("POSITIVE_PEAK_DETECTOR", "Acc_V", 0.5))
features_list.append(mlc_configurator.mlc_feature("NEGATIVE_PEAK_DETECTOR", "Acc_V", 0.5))
features_list.append(mlc_configurator.mlc_feature("MINIMUM", "Acc_V"))
features_list.append(mlc_configurator.mlc_feature("MAXIMUM", "Acc_V"))
features_list.append(mlc_configurator.mlc_feature("MEAN", "Gyr_V"))
features_list.append(mlc_configurator.mlc_feature("VARIANCE", "Gyr_V"))
features_list.append(mlc_configurator.mlc_feature("ENERGY", "Gyr_V"))
features_list.append(mlc_configurator.mlc_feature("PEAK_TO_PEAK", "Gyr_V"))
features_list.append(mlc_configurator.mlc_feature("ZERO_CROSSING", "Gyr_V", 0.5))
features_list.append(mlc_configurator.mlc_feature("POSITIVE_ZERO_CROSSING", "Gyr_V", 0.5))
features_list.append(mlc_configurator.mlc_feature("NEGATIVE_ZERO_CROSSING", "Gyr_V", 0.5))
features_list.append(mlc_configurator.mlc_feature("PEAK_DETECTOR", "Gyr_V", 0.5))
features_list.append(mlc_configurator.mlc_feature("POSITIVE_PEAK_DETECTOR", "Gyr_V", 0.5))
features_list.append(mlc_configurator.mlc_feature("NEGATIVE_PEAK_DETECTOR", "Gyr_V", 0.5))
features_list.append(mlc_configurator.mlc_feature("MINIMUM", "Gyr_V"))
features_list.append(mlc_configurator.mlc_feature("MAXIMUM", "Gyr_V"))
# Some examples of features on filtered inputs: 
#features_list.append(mlc_configurator.mlc_feature("MEAN","Acc_X_filter_1"))
#features_list.append(mlc_configurator.mlc_feature("ENERGY", "Acc_V_filter_2"))
#features_list.append(mlc_configurator.mlc_feature("ZERO_CROSSING", "Acc_V_filter_3", 0.5))

mlc_configurator.arff_generator( device_name = device_name, 
               datalogs = datalogs, 
               results = results, 
               mlc_odr = mlc_odr, 
               input_type = input_type, 
               accelerometer_fs = accelerometer_fs, 
               accelerometer_odr = accelerometer_odr, 
               gyroscope_fs = gyroscope_fs, 
               gyroscope_odr = gyroscope_odr, 
               n_decision_trees = n_decision_trees, 
               window_length = window_length, 
               filters_list = filters_list,  
               features_list = features_list, 
               arff_filename = arff_filename )

## Step 3: Decision Tree generation
The module 'decision_tree_generator.py' is used to generate decision trees through sklearn.

In [ ]:
if (n_decision_trees == 1):
    dectree_accuracy, dectree_nodes = decision_tree_generator.generateDecisionTree(
                                                        arff_filename = arff_filename, 
                                                        dectree_filename = dectree_filenames[0])
else:
    for i in range(n_decision_trees) :
        arff_filename_i = arff_filename + str(i+1)
        decision_tree_generator.generate_subset_of_ARFF( arff_filename = arff_filename, 
                                                         arff_subset_filename = arff_filename_i, 
                                                         classes_subset = result_names[i] )
        print("\n# Decision Tree %d:" %(i+1))
        dectree_accuracy, dectree_nodes = decision_tree_generator.generateDecisionTree( 
                                               arff_filename = arff_filename_i, 
                                               dectree_filename = dectree_filenames[i] )
        

## Step 4: Generation of .ucf file
Settings and files from 'Step 2' and 'Step 3' are used in 'Step 4' to generate the MLC configuration in a .ucf file (a text file containing a sequence of register addresses and corresponding values). 
Meta classifiers, which are filters on the outputs of the decision trees, can be configured here. 
The module 'mlc_configurator.py' is used for this step.

In [ ]:
# Meta-classifiers
# metaclassifierX_values contains the end counter values of the meta classifier associated to the decision tree 'X'
# 4 end counter values are available in LSM6DSOX (the first 4 values in "metaclasifierX_values")
# 8 end counter values are available in LSM6DSRX/ISM330DHCX (the 8 values in "metaclasifierX_values")
# values allowed for end counters are from 0 to 14
metaclassifier1_values = "0,0,0,0,0,0,0,0"
metaclassifier2_values = "0,0,0,0,0,0,0,0"
metaclassifier3_values = "0,0,0,0,0,0,0,0"
metaclassifier4_values = "0,0,0,0,0,0,0,0"
metaclassifier5_values = "0,0,0,0,0,0,0,0"
metaclassifier6_values = "0,0,0,0,0,0,0,0"
metaclassifier7_values = "0,0,0,0,0,0,0,0"
metaclassifier8_values = "0,0,0,0,0,0,0,0"
metaclassifier_values = [metaclassifier1_values, metaclassifier2_values, metaclassifier3_values, metaclassifier4_values, metaclassifier5_values, metaclassifier6_values, metaclassifier7_values, metaclassifier8_values]

mlc_configurator.ucf_generator( device_name = device_name, 
                                arff_filename = arff_filename, 
                                dectree_filenames = dectree_filenames,
                                result_names = result_names, 
                                result_values = result_values, 
                                metaclassifier_values = metaclassifier_values, 
                                ucf_filename = ucf_filename )

## (Optional) Testing decision tree
This section allows to test the .arff file on the decision tree.
The .arff file can be generated in Step2 after selecting data patterns (in Step1) and Features (in Step2). 
The module 'mlc_test.py' is used for this step.

In [ ]:
# Testing
testresults_filename = os.path.join(current_directory, "test_results.txt")
mlc_test.test_arff_on_decisiontree( dectree_filename = dectree_filenames[0], 
                                    arff_filename = arff_filename, 
                                    testresults_filename = testresults_filename )

# show test results:
file_o = open(testresults_filename)   
content = file_o.read()                
print(content)                      
file_o.close()